In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import StratifiedShuffleSplit

from threading import Thread, Lock

/Users/danilo/.conda/envs/bcb/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


## Dataset Configuration

In [2]:
ds = pd.read_csv(os.path.join('data', 'toyDataset.csv')).values
label = pd.read_csv(os.path.join('data', 'toyLabel.csv')).values.reshape(-1)
ds_names = ['ds1', 'ds2', 'ds3']

In [3]:
def random_sampling(X, y, n_samples):
    return next(StratifiedShuffleSplit(n_splits=1, test_size=n_samples).split(X, y))

In [4]:
tr_idx, ts_idx = random_sampling(ds, label, n_samples=0.25)

ds1 = np.hstack([ds[:,:5], ds[:,10:15]])
ds1_tr = ds1[tr_idx]
ds1_ts = ds1[ts_idx]

ds2 = np.hstack([ds[:,5:10], ds[:,15:20]])
ds2_tr = ds2[tr_idx]
ds2_ts = ds2[ts_idx]

ds3 = ds[:,20:]
ds3_tr = ds3[tr_idx]
ds3_ts = ds3[ts_idx]

y_t_ = label[tr_idx]
y_t_test = label[ts_idx]

ds_list_complete = [ds1, ds2, ds3]
ds_list = [ds1_tr, ds2_tr, ds3_tr]
ds_test = [ds1_ts, ds2_ts, ds3_ts]

## Kernel Definition

In [5]:
kernel_names_0 = ['linear', 'polynomial', 'gaussian']
kernel_type_0 = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]

In [6]:
kernel_names_1 = ['laplacian', 'polynomial', 'gaussian']
kernel_type_1 = [{'laplacian':[0.1, 0.2, 0.3], 'polynomial':[2, 3], 'gaussian':[0.2, 0.3]},
               {'laplacian':[0.5, 0.7], 'polynomial':[5, 8], 'gaussian':[0.5, 0.7]}]

In [7]:
kernel_names_2 = ['sigmoid', 'polynomial', 'gaussian']
kernel_type_2 = [{'sigmoid':[0.1, 0.2, 0.3], 'polynomial':[2, 3], 'gaussian':[0.2, 0.3]},
               {'sigmoid':[0.5, 0.7], 'polynomial':[5, 8], 'gaussian':[0.5, 0.7]}]

In [8]:
kernel_names_3 = ['laplacian', 'gaussian']
kernel_type_3 = [{'laplacian':[0.2, 0.6], 'gaussian':[0.3, 0.6]},
               {'laplacian':[0.4, 0.9], 'gaussian':[0.5, 1]}]

In [9]:
kernel_names_4 = ['linear', 'gaussian']
kernel_type_4 = [{'linear':[1], 'gaussian':[0.3, 0.6]},
               {'linear':[1], 'gaussian':[0.5, 1]}]

In [10]:
kernel_names_5 = ['polynomial', 'gaussian']
kernel_type_5 = [{'polynomial':[2, 7], 'gaussian':[0.3, 0.6]},
               {'polynomial':[3, 5], 'gaussian':[0.5, 1]}]

In [11]:
kernel_names_6 = ['sigmoid', 'gaussian']
kernel_type_6 = [{'sigmoid':[0.2, 0.6], 'gaussian':[0.3, 0.6]},
               {'sigmoid':[0.4, 0.9], 'gaussian':[0.5, 1]}]

## Other shared parameters initialization

In [12]:
estimator = ca.centeredKernelAlignment
lock_toy = Lock()
valid_fold = 3
threads = []
pen_params = [0.5, 0.7, 0.9, 1.3]

## Thread

In [13]:
def child(sampler, estimator, ds_list, ds_test ,y_t_, y_t_test, lock_toy, valid_fold, exclusion_list, verbose, approach):

    #TOY
    
    # Linear, Polynomial, Gaussian
    result1 = sampler.sample(kernel_type_0, estimator, ds_list, y_t_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_0)
    ut.testConfigurations(estimator, y_t_, y_t_test, w_list, ds_list, ds_test, kernel_names_0, lamb_list, sparsity, 'classification', lock_toy, fileToWrite = 'results_temp/Toy_test.txt', header = 'Toy Linear - Polynomial - Gaussian\n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Toy_train.txt', header='\nToy Linear - Polynomial - Gaussian\n' + approach + '\n', lock = lock_toy)
    
    # Laplacian, Polynomial, Gaussian
    result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_t_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_1)
    ut.testConfigurations(estimator, y_t_, y_t_test, w_list, ds_list, ds_test, kernel_names_1, lamb_list, sparsity, 'classification', lock_toy, fileToWrite = 'results_temp/Toy_test.txt', header = 'Toy Laplacian - Polynomial - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Toy_train.txt', header='\nToy Laplacian - Polynomial - Gaussian\n' + approach + '\n', lock = lock_toy)
    
    # Sigmoid, Polynomial, Gaussian
    result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_t_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_2)
    ut.testConfigurations(estimator, y_t_, y_t_test, w_list, ds_list, ds_test, kernel_names_2, lamb_list, sparsity, 'classification', lock_toy, fileToWrite = 'results_temp/Toy_test.txt', header = 'Toy Sigmoid - Polynomial - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Toy_train.txt', header='\nToy Sigmoid - Polynomial - Gaussian\n' + approach + '\n', lock = lock_toy)
    
    #Laplacian - Gaussian
    result1 = sampler.sample(kernel_type_3, estimator, ds_list, y_t_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_3)
    ut.testConfigurations(estimator, y_t_, y_t_test, w_list, ds_list, ds_test, kernel_names_3, lamb_list, sparsity, 'classification', lock_toy, fileToWrite = 'results_temp/Toy_test.txt', header = 'Toy Laplacian - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Toy_train.txt', header='\nToy Laplacian - Gaussian\n' + approach + '\n', lock = lock_toy)
    
    #Linear - Gaussian
    result1 = sampler.sample(kernel_type_4, estimator, ds_list, y_t_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_4)
    ut.testConfigurations(estimator, y_t_, y_t_test, w_list, ds_list, ds_test, kernel_names_4, lamb_list, sparsity, 'classification', lock_toy, fileToWrite = 'results_temp/Toy_test.txt', header = 'Toy Linear - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Toy_train.txt', header='\nToy Linear - Gaussian\n' + approach + '\n', lock = lock_toy)
    
    #Polynomial - Gaussian
    result1 = sampler.sample(kernel_type_5, estimator, ds_list, y_t_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_5)
    ut.testConfigurations(estimator, y_t_, y_t_test, w_list, ds_list, ds_test, kernel_names_5, lamb_list, sparsity, 'classification', lock_toy, fileToWrite = 'results_temp/Toy_test.txt', header = 'Toy Polynomial - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Toy_train.txt', header='\nToy Polynomial - Gaussian\n' + approach + '\n', lock = lock_toy)
    
    #Sigmoid - Gaussian
    result1 = sampler.sample(kernel_type_6, estimator, ds_list, y_t_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_6)
    ut.testConfigurations(estimator, y_t_, y_t_test, w_list, ds_list, ds_test, kernel_names_6, lamb_list, sparsity, 'classification', lock_toy, fileToWrite = 'results_temp/Toy_test.txt', header = 'Toy Sigmoid - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Toy_train.txt', header='\nToy Sigmoid - Gaussian\n' + approach + '\n', lock = lock_toy)
    

## L2 Penalty, Centering, Normalization

In [14]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [15]:
t = Thread(target=child, args=([sampler, estimator, ds_list, ds_test, y_t_, y_t_test, lock_toy, valid_fold, None, False, 'L2 - Centering - Normalizing']))
t.start()
threads.append(t)

## L2 Penalty, Centering, K-Normalization

In [16]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=0.25, sparsity = False, centering = True, normalize_kernels = False)

In [17]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_t_, y_t_test, lock_toy, valid_fold, None, False, 'L2 - Centering - K Normalizing'))
t.start()
threads.append(t)

In [ ]:
for t in threads:
    t.join()

print("Operations completed")

## L1 Penalty, Centering, Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=([sampler, estimator, ds_list, ds_test, y_t_, y_t_test, lock_toy, valid_fold, None, False, 'L1 - Centering - Normalizing']))
t.start()
threads.append(t)

## L1 Penalty, Centering, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalize_kernels = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_t_, y_t_test, lock_toy, valid_fold, None, False, 'L1 - Centering - K Normalizing'))
t.start()
threads.append(t)

In [ ]:
for t in threads:
    t.join()

print("Operations completed")